In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np

tfe.enable_eager_execution()

In [2]:
language = 'fra'
data_path = f'../datasets/Bilingual_Sentence_Pairs/{language}.txt' # Path to the data txt file on disk.

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [3]:
print(lines[2])

Run!	Cours !


In [4]:
num_samples = 1500  # Number of samples to train on.

input_texts = []
target_texts = []
input_words = set()
target_words = set()
max_encoder_seq_length = 0  # T_encoder
max_decoder_seq_length = 0  # T_decoder

for line in lines[: min(num_samples, len(lines)-1)]:
    cur_input_text, cur_target_text = line.split('\t')
    cur_target_text = '<START> ' + cur_target_text + ' <END>'
    
    input_texts.append(cur_input_text)
    target_texts.append(cur_target_text)
    
    max_encoder_seq_length = max(max_encoder_seq_length, len(cur_input_text))
    max_decoder_seq_length = max(max_decoder_seq_length, len(cur_target_text))
    
    for word in cur_input_text.split():
        if word not in input_words:
            input_words.add(word)
    
    for word in cur_target_text.split():
        if word not in target_words:
            target_words.add(word)

input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_words = len(input_words)  # V_encoder
num_decoder_words = len(target_words)  # V_decoder
input_word_to_index = dict([(char, i) for i, char in enumerate(input_words)])
target_word_to_index = dict([(char, i) for i, char in enumerate(target_words)])

In [5]:
print('Number of samples:', len(input_texts))
print('Number of unique input words:', num_encoder_words)
print('Number of unique output words:', num_decoder_words)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 1500
Number of unique input words: 670
Number of unique output words: 1265
Max sequence length for inputs: 11
Max sequence length for outputs: 47


In [6]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_words), dtype=np.float32)  # shape: N x T_encoder x V_encoder
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_words), dtype=np.float32)  # shape: N x T_decoder x V_decoder
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_words), dtype=np.float32) # shape: N x T_decoder x V_decoder

for i, (cur_input_text, cur_target_text) in enumerate(zip(input_texts, target_texts)):
    
    for t, word in enumerate(cur_input_text.split()):
        encoder_input_data[i, t, input_word_to_index[word]] = 1
    
    for t, word in enumerate(cur_target_text.split()):
        decoder_input_data[i, t, target_word_to_index[word]] = 1
        
        if t > 0:
            decoder_target_data[i, t-1, target_word_to_index[word]] = 1

In [13]:
latent_dim = 128  # Latent dimensionality of the encoding space.

# configure encoder
encoder_inputs = tf.keras.layers.Input(shape=(None, num_encoder_words))
encoder = tf.keras.layers.CuDNNLSTM(units=latent_dim, return_state=True)
encoder_outputs, encoder_h, encoder_c = encoder(encoder_inputs)
encoder_state = [encoder_h, encoder_c]

# configure decoder
decoder_inputs = tf.keras.layers.Input(shape=(None, num_decoder_words))
decoder = tf.keras.layers.CuDNNLSTM(units=latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder(decoder_inputs, initial_state=encoder_state)

# configure dense layer
dense_layer = tf.keras.layers.Dense(units=num_decoder_words, activation='softmax')
dense_outputs = dense_layer(decoder_outputs)

<DeferredTensor 'None' shape=(?, ?, 1265) dtype=float32>

In [18]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.

model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=dense_outputs)
model.compile(optimizer=tf.train.RMSPropOptimizer(0.005), loss='categorical_crossentropy')
model.fit(x=[encoder_input_data, decoder_input_data], y=decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)


Epoch 1/100
19/19 [==============================] - 2s 113ms/step - loss: 0.5896 - val_loss: 0.5921
Epoch 2/100
19/19 [==============================] - 2s 100ms/step - loss: 0.5894 - val_loss: 0.5919
Epoch 3/100
19/19 [==============================] - 2s 102ms/step - loss: 0.5889 - val_loss: 0.5911
Epoch 4/100
19/19 [==============================] - 2s 100ms/step - loss: 0.5877 - val_loss: 0.5894
Epoch 5/100
19/19 [==============================] - 2s 98ms/step - loss: 0.5852 - val_loss: 0.5860
Epoch 6/100
19/19 [==============================] - 2s 97ms/step - loss: 0.5799 - val_loss: 0.5772
Epoch 7/100
19/19 [==============================] - 2s 99ms/step - loss: 0.5343 - val_loss: 0.4496
Epoch 8/100
19/19 [==============================] - 2s 99ms/step - loss: 0.4156 - val_loss: 0.4033
Epoch 9/100
19/19 [==============================] - 2s 98ms/step - loss: 0.3915 - val_loss: 0.4075
Epoch 10/100
19/19 [==============================] - 2s 99ms/step - loss: 0.3787 - val_loss: 0.

In [19]:
sampling_encoder_model = tf.keras.Model(inputs=encoder_inputs, outputs=encoder_state)

sampling_decoder_input_state_h = tf.keras.layers.Input(shape=(latent_dim, ))
sampling_decoder_input_state_c = tf.keras.layers.Input(shape=(latent_dim, ))
sampling_decoder_input_state = [sampling_decoder_input_state_h, sampling_decoder_input_state_c]
sampling_decoder_outputs, sampling_decoder_state_h, sampling_decoder_state_c = decoder(decoder_inputs, initial_state=sampling_decoder_input_state)
sampling_decoder_output_state = [sampling_decoder_state_h, sampling_decoder_state_c]
sampling_dense_outputs = dense_layer(sampling_decoder_outputs)

sampling_decoder_model = tf.keras.Model(inputs=[decoder_inputs]+sampling_decoder_input_state, outputs=[sampling_dense_outputs]+sampling_decoder_output_state)

In [20]:
def decode_sequence(input_seq):
    states_value = sampling_encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1, 1, num_decoder_words), dtype=np.float32)
    target_seq[0, 0, target_word_to_index['<START>']] = 1
    
    stop_codition = False
    decoded_words_array = []
    
    while not stop_codition:
#         print("target_seq shape: {}".format(target_seq.dtype))
#         print("states_value shape: {}".format(states_value.dtype))
        output_words, cur_state_h, cur_state_c = sampling_decoder_model([target_seq] + states_value)
        
        sampled_word_index = tf.argmax(output_words[0, -1, :])
        sampled_word = target_words[sampled_word_index]
        decoded_words_array.append(sampled_word)
        
        if sampled_word == '<END>' or len(decoded_words_array) > max_decoder_seq_length:
            stop_codition = True
        
        target_seq = np.zeros_like(target_seq, dtype=np.float32)
        target_seq[0, 0, sampled_word_index] = 1
        states_value = [cur_state_h, cur_state_c]
    
    return " ".join(decoded_words_array)

In [21]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('---------------------------------------')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

---------------------------------------
Input sentence: Go.
Decoded sentence: Venez à nous. <END>
---------------------------------------
Input sentence: Hi.
Decoded sentence: Réveille-toi ! <END>
---------------------------------------
Input sentence: Run!
Decoded sentence: Réveille-toi ! <END>
---------------------------------------
Input sentence: Run!
Decoded sentence: Réveille-toi ! <END>
---------------------------------------
Input sentence: Wow!
Decoded sentence: Ne parle pas ! <END>
---------------------------------------
Input sentence: Fire!
Decoded sentence: Venez à moi. <END>
---------------------------------------
Input sentence: Help!
Decoded sentence: À la bonne heure ! <END>
---------------------------------------
Input sentence: Jump.
Decoded sentence: Fais-moi confiance. <END>
---------------------------------------
Input sentence: Stop!
Decoded sentence: Fais attention ! <END>
---------------------------------------
Input sentence: Stop!
Decoded sentence: Fais atten

---------------------------------------
Input sentence: Be nice.
Decoded sentence: Soyez prudente ! <END>
---------------------------------------
Input sentence: Be nice.
Decoded sentence: Soyez prudente ! <END>
---------------------------------------
Input sentence: Be nice.
Decoded sentence: Soyez prudente ! <END>
---------------------------------------
Input sentence: Be nice.
Decoded sentence: Soyez prudente ! <END>
---------------------------------------
Input sentence: Be nice.
Decoded sentence: Soyez prudente ! <END>
---------------------------------------
Input sentence: Be nice.
Decoded sentence: Soyez prudente ! <END>
---------------------------------------
Input sentence: Beat it.
Decoded sentence: Va voir ailleurs si j'y suis ! <END>
---------------------------------------
Input sentence: Call me.
Decoded sentence: Recule ! <END>
---------------------------------------
Input sentence: Call me.
Decoded sentence: Recule ! <END>
---------------------------------------
Input se